In [2]:
import pandas as pd
import numpy as np
import os

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [64]:
# Set up paths
raw_folder = './raw_data/'
tmp_folder = './tmp_data/'
processed_folder = './processed_data/'

imdb_rating_folder = raw_folder + 'imdb/'
cmu_folder = raw_folder + 'cmu/'
baby_names_folder = raw_folder + 'baby_names_national/'

# Data preprocessing
In this notebook we aim to select the relevant features for our model and to clean the data. This also includes merging content from the different datasets, from various sources such as the `IMDB` datasets, the baby names datasets, etc... which have been either downloaded or scraped from the web.

**Note on structure:** We separate dataframes into various folders, raw, tmp (temporary data) and processed. 
- The raw data is what we download from the web, including the initial CMU datasets. These are unmodified content that we use as a basis for our work, it is read-only.
- The temporary (tmp) data is the data that we use to work on, and that we modify. This is often the case for scrapped data from the web, or dataframes that are being used in a different notebook other than processing.
- We put our final cleaned data in the processed folder, which is the data that we use for our models.

## 1. Initial view on the data
### a. CMU datasets
Our project requires the use of the `character.metadata.tsv` and `movie.metadata.tsv` files from the CMU databases.


In [12]:
# Import character metadata
character_df = pd.read_csv(cmu_folder + 'character.metadata.tsv', sep='\t', header=None)

# Add column names deduced from README
character_df.columns = ['wiki_ID', 'free_ID', 'release', 'char_name', 'DOB', 'gender', 'height', 'ethnicity', 'act_name', 'age_at_release', 'free_char_map1', 'free_char_map2', 'free_char_map3']

display(character_df.head())
print(f"There are {len(character_df)} characters in the dataset.")

,wiki_ID,free_ID,release,char_name,DOB,gender,height,ethnicity,act_name,age_at_release,free_char_map1,free_char_map2,free_char_map3
0,975900,/m/03vyhn,2001-08-24,Akooshay,1958-08-26,F,1.620,NaN,Wanda De Jesus,42.0,/m/0bgchxw,/m/0bgcj3x,/m/03wcfv7
1,975900,/m/03vyhn,2001-08-24,Lieutenant Melanie Ballard,1974-08-15,F,1.780,/m/044038p,Natasha Henstridge,27.0,/m/0jys3m,/m/0bgchn4,/m/0346l4
2,975900,/m/03vyhn,2001-08-24,Desolation Williams,1969-06-15,M,1.727,/m/0x67,Ice Cube,32.0,/m/0jys3g,/m/0bgchn_,/m/01vw26l
3,975900,/m/03vyhn,2001-08-24,Sgt Jericho Butler,1967-09-12,M,1.750,NaN,Jason Statham,33.0,/m/02vchl6,/m/0bgchnq,/m/034hyc
4,975900,/m/03vyhn,2001-08-24,Bashira Kincaid,1977-09-25,F,1.650,NaN,Clea DuVall,23.0,/m/02vbb3r,/m/0bgchp9,/m/01y9xg


There are 450669 characters in the dataset.


In [14]:
# Import movie metadata
movie_df = pd.read_csv(cmu_folder + 'movie.metadata.tsv', sep='\t', header=None)

# Add column names deduced from README
movie_df.columns = ['wiki_ID', 'free_ID', 'mov_name', 'release', 'revenue', 'runtime', 'languages', 'countries', 'genres']
display(movie_df.head(5))
print(f"There are {len(movie_df)} movies in the dataset.")
print(f"Is the wiki_ID unique? {movie_df.wiki_ID.is_unique}")

,wiki_ID,free_ID,mov_name,release,revenue,runtime,languages,countries,genres
0,975900,/m/03vyhn,Ghosts of Mars,2001-08-24,14010832.0,98.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/01jfsb"": ""Thriller"", ""/m/06n90"": ""Science..."
1,3196793,/m/08yl5d,Getting Away with Murder: The JonBenét Ramsey ...,2000-02-16,NaN,95.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/02n4kr"": ""Mystery"", ""/m/03bxz7"": ""Biograp..."
2,28463795,/m/0crgdbh,Brun bitter,1988,NaN,83.0,"{""/m/05f_3"": ""Norwegian Language""}","{""/m/05b4w"": ""Norway""}","{""/m/0lsxr"": ""Crime Fiction"", ""/m/07s9rl0"": ""D..."
3,9363483,/m/0285_cd,White Of The Eye,1987,NaN,110.0,"{""/m/02h40lc"": ""English Language""}","{""/m/07ssc"": ""United Kingdom""}","{""/m/01jfsb"": ""Thriller"", ""/m/0glj9q"": ""Erotic..."
4,261236,/m/01mrr1,A Woman in Flames,1983,NaN,106.0,"{""/m/04306rv"": ""German Language""}","{""/m/0345h"": ""Germany""}","{""/m/07s9rl0"": ""Drama""}"


There are 81741 movies in the dataset.
Is the wiki_ID unique? True


Great, we can use the wikipedia IDs as unique identifers for the movies!

We immediately see that there are some columns that we won't need in our analysis, which we will drop later on, such as the freebase character maps, releases in the characters dataframe (because they are already in the movies dataframe), etc...

We also see that there are some missing values for `revenues`, we need to take note of how any of them are missing to see if they would affect our analysis.

In [19]:
nb_revenue_missing = movie_df['revenue'].isna().sum()
total_movies = len(movie_df)
perc_missing = (nb_revenue_missing / total_movies)*100
print(f"Percentage of missing values in column 'revenue': {perc_missing:.2f}%")

Percentage of missing values in column 'revenue': 89.72%


### b. IMDB datasets: Ratings
We would like to use ratings and rating counts to assess the popularity of a movie. We will use the `imdb_ratings.tsv` file from the [IMDB datasets](https://datasets.imdbws.com/).

In [15]:
# Import imdb data
rating_df = pd.read_csv(imdb_rating_folder + 'imdb_rating.tsv', sep='\t', index_col='tconst')
display(rating_df.head())

,averageRating,numVotes
tconst,,
tt0000001,5.7,2004
tt0000002,5.8,269
tt0000003,6.5,1904
tt0000004,5.5,178
tt0000005,6.2,2685


In [16]:
# Verify the indexes are unique
print(f"Is the indexing unique ? {rating_df.index.is_unique}")

Is the indexing unique ? True


`tconst` is a unique identifer in the IMDB database, which we can use to merge with our other dataframes. We need to figure out how to merge these dataframes together.

### c. Baby names data
We consider the names of babies across the years in the USA to be a good indicator of the popularity of a name. We will use the `NationalNames.csv` file from the [US Social Security Administration](https://www.ssa.gov/oact/babynames/limits.html), and will be put in the `baby_names_national` folder. This dataset has all the names of babies born in the USA from 1880 to 2023, where each count is the number of babies born with that name in that year.

We need to iterate over all the years to create a dataframe with the counts of all the names, for all years:
- `name`: name of the baby
- `year`: year of birth
- `number`: number of babies born with that name in that year 
- `percentage`: percentage of babies born with that name in that year, for normalization purposes (added later in the notebook)

In [18]:
# Define the path to your dataset folder
folder_path = 'raw_data/baby_names_national/'

# Create an empty list to store individual DataFrames
data_frames = []

# Iterate through each file in the folder
for filename in os.listdir(folder_path):
    if filename.startswith('yob') and filename.endswith('.txt'):
        # Extract the year from the filename
        year = int(filename[3:-4])

        # Read the data from the file into a DataFrame
        file_path = os.path.join(folder_path, filename)
        df = pd.read_csv(file_path, header=None, names=['name', 'gender', 'number'])

        # Add the 'year' column to the DataFrame
        df['year'] = year

        # Append the current DataFrame to the list
        data_frames.append(df)

# Concatenate all DataFrames in the list into one DataFrame
baby_name_df = pd.concat(data_frames, ignore_index=True)

# Set the 'name' column as the index
# combined_data.set_index('name', inplace=True)

# Display the resulting DataFrame
baby_name_df.head()

,name,gender,number,year
0,Emily,F,25957,2000
1,Hannah,F,23085,2000
2,Madison,F,19968,2000
3,Ashley,F,17998,2000
4,Sarah,F,17712,2000


## 3. Data processing and cleaning
In this section we modify some of the dataframes values to make them usable for our analysis, and populate them with extra data from other sources.

### 3.a Linking external movie IDs (Scraping part 1)
One of our main sources of data is the [IMDB](https://www.imdb.com/) and the [TMDB](https://www.themoviedb.org/) website. To be able to do so, we need to find a link from our CMU IDs (wikipedia) to the IMDB and TMDB movie IDs. We've explored 2 methods of collecting this kind of data. All the process is extensively documented in the `convert_ds.ipynb` notebook. 

Therefore make sure you run [this notebook](./scraping/convert_ids.ipynb) before continuing. It should have generated the `movies_external_ids.csv` file in the `tmp` folder, which we will use in the following steps.

In [24]:
# Test import with a try
try:
    df = pd.read_csv(tmp_folder + 'movies_external_ids.csv', sep='\t', header=None)
except:
    raise FileNotFoundError("Please run the ID scrapping notebook first.")

### 3.b Movie metadata preprocessing
#### Release date column
The release date column is very important for our study. Let's compute the missing values and remove the NaN value in the release column, we will drop these rows later on.

# <h1 style="font-size: 100px; color: red">⚠️**TODO:** change structure instead of `release` to `year` and `month` columns. If no month, just leave nan!</h1>

In [31]:
nb_release_missing = movie_df['release'].isna().sum()
total_movies = len(movie_df)
perc_missing = (nb_release_missing / total_movies)*100
print(f"Percentage of missing values in column 'release': {perc_missing:.2f}%")

Percentage of missing values in column 'release': 8.44%


. The movies release has a weird format. There are 3 different formats:
- of length 10: `YYYY-MM-DD`
- of length 7: `YYYY-MM`
- of length 4: `YYYY`

In [33]:
counts_lengths = movie_df['release'].dropna().apply(lambda x: len(str(x))).value_counts()
print(f"In the release column of movie_df the string have the following lengths with their frequency : \n\n{counts_lengths}")

In the release column of movie_df the string have the following lengths with their frequency : 

release
10    39373
4     32172
7      3294
Name: count, dtype: int64


In [34]:
length4_test = movie_df[movie_df['release'].apply(lambda x: len(str(x)) == 4)].iloc[0]['release']
print(f"Example of a value of length 4: {length4_test}")
length7_test = movie_df[movie_df['release'].apply(lambda x: len(str(x)) == 7)].iloc[0]['release']
print(f"Example of a value of length 7: {length7_test}")
length10_test = movie_df[movie_df['release'].apply(lambda x: len(str(x)) == 10)].iloc[0]['release']
print(f"Example of a value of length 10: {length10_test}")

Example of a value of length 4: 1988
Example of a value of length 7: 2003-10
Example of a value of length 10: 2001-08-24


We should convert the release date to keep only the year as we have only the year in the baby name dataset. To do so we simply take the first 4 characters of the release date, which is the year. We then convert it to an integer.

In [60]:
# Drop rows with NaN values in the release column
movie_datetime_df = movie_df.dropna(subset=['release']).copy(deep=True)

# Take only year
movie_datetime_df['release'] = movie_datetime_df['release'].apply(lambda x: str(x)[:4]).astype(np.int64)

# Remove outliers (years before 1800, there is one that is at 1010)
movie_datetime_df = movie_datetime_df[movie_datetime_df['release'] >= 1800]

movie_datetime_df[['wiki_ID', 'mov_name', 'release']].head()

,wiki_ID,mov_name,release
0,975900,Ghosts of Mars,2001
1,3196793,Getting Away with Murder: The JonBenét Ramsey ...,2000
2,28463795,Brun bitter,1988
3,9363483,White Of The Eye,1987
4,261236,A Woman in Flames,1983


In [63]:
# Reset the movie_df to the datetime version
movie_df = movie_datetime_df.copy(deep=True)

#### Add movie ratings and count

In [68]:
# Import idmb with wikidata index mapping
movies_to_imdb_id_df = pd.read_csv(tmp_folder + 'movies_external_ids.csv')
display(movies_to_imdb_id_df)

,wikipedia_ID,IMDB_ID,TMDB_ID
0,22760465,ch0112831,NaN
1,185519,nm0158560,NaN
2,12754957,nm0704910,NaN
3,10111255,nm1016169,NaN
4,7669437,tt0000001,16612.0
...,...,...,...
76650,22780907,tt9834564,834616.0
76651,7316179,tt9855214,279530.0
76652,17475562,tt9881364,1181096.0
76653,8783319,tt9898504,NaN


Let's check if we have the rating of all the movies. To do so we carry out a left merge and count the number of missing values:

In [75]:
left_merged_imdb_movie_df = pd.merge(movie_df.reset_index(), movies_to_imdb_id_df, left_on='wiki_ID', right_on='wikipedia_ID', how='left').copy(deep=True)
display(left_merged_imdb_movie_df.head(2))
print(f"Length of the dataframe : {len(left_merged_imdb_movie_df)}")

,index,wiki_ID,free_ID,mov_name,release,revenue,runtime,languages,countries,genres,wikipedia_ID,IMDB_ID,TMDB_ID
0,0,975900,/m/03vyhn,Ghosts of Mars,2001,14010832.0,98.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/01jfsb"": ""Thriller"", ""/m/06n90"": ""Science...",975900.0,tt0228333,10016.0
1,1,3196793,/m/08yl5d,Getting Away with Murder: The JonBenét Ramsey ...,2000,NaN,95.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/02n4kr"": ""Mystery"", ""/m/03bxz7"": ""Biograp...",3196793.0,tt0245916,784579.0


Length of the dataframe : 74838


In [79]:
nb_rating_missing = left_merged_imdb_movie_df['IMDB_ID'].isna().sum()
total_movies = len(left_merged_imdb_movie_df)
perc_missing = (nb_rating_missing / total_movies)*100
print(f"Percentage of missing values in column 'averageRating': {perc_missing:.2f}%")

Percentage of missing values in column 'averageRating': 5.04%


We miss about 5% of the IMDB index. Therefore, the number of movie is reduced of 5% as well. Let's make the inner merge to keep only the movie for which we know the rating

In [84]:
merged_imdb_movie_df = pd.merge(movie_df.reset_index(), movies_to_imdb_id_df, left_on='wiki_ID', right_on='wikipedia_ID', how='inner').copy(deep=True)
display(merged_imdb_movie_df.head(2))
print(f"Length of the dataframe : {len(merged_imdb_movie_df)}")

merged_rating_movie_df = pd.merge(merged_imdb_movie_df, rating_df, left_on='IMDB_ID', right_on='tconst', how='inner').copy(deep=True)
display(merged_rating_movie_df.head(2))
print(f"Length of the dataframe : {len(merged_rating_movie_df)}")

,index,wiki_ID,free_ID,mov_name,release,revenue,runtime,languages,countries,genres,wikipedia_ID,IMDB_ID,TMDB_ID
0,0,975900,/m/03vyhn,Ghosts of Mars,2001,14010832.0,98.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/01jfsb"": ""Thriller"", ""/m/06n90"": ""Science...",975900,tt0228333,10016.0
1,1,3196793,/m/08yl5d,Getting Away with Murder: The JonBenét Ramsey ...,2000,NaN,95.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/02n4kr"": ""Mystery"", ""/m/03bxz7"": ""Biograp...",3196793,tt0245916,784579.0


Length of the dataframe : 71064


,index,wiki_ID,free_ID,mov_name,release,revenue,runtime,languages,countries,genres,wikipedia_ID,IMDB_ID,TMDB_ID,averageRating,numVotes
0,0,975900,/m/03vyhn,Ghosts of Mars,2001,14010832.0,98.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/01jfsb"": ""Thriller"", ""/m/06n90"": ""Science...",975900,tt0228333,10016.0,4.9,56880
1,1,3196793,/m/08yl5d,Getting Away with Murder: The JonBenét Ramsey ...,2000,NaN,95.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/02n4kr"": ""Mystery"", ""/m/03bxz7"": ""Biograp...",3196793,tt0245916,784579.0,6.0,69


Length of the dataframe : 65503


Lets remove the columns that we don't need

In [88]:
movie_rating_df = merged_rating_movie_df[['wiki_ID', 'mov_name', 'release', 'revenue', 'genres', 'numVotes', 'averageRating']].copy(deep=True)
display(movie_rating_df.head(2))
print(f"length of the dataframe : {len(movie_rating_df)}")

,wiki_ID,mov_name,release,revenue,genres,numVotes,averageRating
0,975900,Ghosts of Mars,2001,14010832.0,"{""/m/01jfsb"": ""Thriller"", ""/m/06n90"": ""Science...",56880,4.9
1,3196793,Getting Away with Murder: The JonBenét Ramsey ...,2000,NaN,"{""/m/02n4kr"": ""Mystery"", ""/m/03bxz7"": ""Biograp...",69,6.0


length of the dataframe : 65503


### Genre column cleaning

In [ ]:
print(f"Are there movies with missing genre fields? {movies_titles['genres'].isna().sum() != 0}")

In the genre column we have dictionaries. Let's keep only the name of the genre

In [84]:
# Get unique values (excluding NaN)
unique_values = movie_df['genres'].unique()

# Print the unique values
print("Unique values:", unique_values)

Unique values: ['{"/m/01jfsb": "Thriller", "/m/06n90": "Science Fiction", "/m/03npn": "Horror", "/m/03k9fj": "Adventure", "/m/0fdjb": "Supernatural", "/m/02kdv5l": "Action", "/m/09zvmj": "Space western"}'
 '{"/m/02n4kr": "Mystery", "/m/03bxz7": "Biographical film", "/m/07s9rl0": "Drama", "/m/0hj3n01": "Crime Drama"}'
 '{"/m/0lsxr": "Crime Fiction", "/m/07s9rl0": "Drama"}' ...
 '{"/m/0lsxr": "Crime Fiction", "/m/01jfsb": "Thriller", "/m/01z4y": "Comedy", "/m/0fdjb": "Supernatural"}'
 '{"/m/01z02hx": "Sports", "/m/0lsxr": "Crime Fiction", "/m/02kdv5l": "Action", "/m/07s9rl0": "Drama"}'
 '{"/m/06n90": "Science Fiction", "/m/0gw5n2f": "Japanese Movies", "/m/03k9fj": "Adventure", "/m/0hcr": "Animation", "/m/02hmvc": "Short Film", "/m/0jxy": "Anime", "/m/07s9rl0": "Drama"}']


TODO : preprocess the genres in order to keep only the values and discard the freebase key

## Baby name data preprocessing

Combine the rows with the same name and same year in order to ignore the gender in the dataset

In [14]:
baby_name_filtered_df = baby_name_df.groupby(['name', 'year'])['number'].sum().to_frame()

display(baby_name_filtered_df.head())

number
name  year        
Aaban 2007       5
      2009       6
      2010       9
      2011      11
      2012      11

For each datapoint, compute the percentage of the total births in the year

In [15]:
# group the baby name dataframe by year to get the number of birth per year.
birth_per_year_df = baby_name_filtered_df.groupby('year')['number'].sum().to_frame()
birth_per_year_df.reset_index(inplace=True)

birth_per_year_df = birth_per_year_df.rename(columns={'number': 'total_number'})

print("birth_per_year_df:")
display(birth_per_year_df.head())


# Merge dataframes
merged_df = pd.merge(baby_name_filtered_df.reset_index(), birth_per_year_df, on='year')

# Calculate the percentage and add it as a new column to dataframe
merged_df['percentage'] = (merged_df['number'] / merged_df['total_number']) * 100

print("merged_df:")
display(merged_df.head())

baby_name_with_percentage_df = merged_df.drop('total_number', axis=1)
baby_name_with_percentage_df.set_index(['name', 'year'], inplace=True)

print("baby_name_with_percentage_df:")
display(baby_name_with_percentage_df.head())


birth_per_year_df:


,year,total_number
0,1880,201484
1,1881,192690
2,1882,221533
3,1883,216945
4,1884,243461


merged_df:


,name,year,number,total_number,percentage
0,Aaban,2007,5,3996648,0.000125
1,Aadam,2007,8,3996648,0.000200
2,Aadan,2007,8,3996648,0.000200
3,Aadarsh,2007,13,3996648,0.000325
4,Aaden,2007,157,3996648,0.003928


baby_name_with_percentage_df:


,,number,percentage
name,year,,
Aaban,2007,5,0.000125
Aadam,2007,8,0.000200
Aadan,2007,8,0.000200
Aadarsh,2007,13,0.000325
Aaden,2007,157,0.003928


## Character metadata preprocessing

The character dataframe has not a unique index. It is due to the several Nan values present in the same movie. To tackles this we can drop the rows that have a NaN as name and see if it solve the issue

In [16]:
character_filtered = character_df.copy(deep=True)

# drop the rows with a NaN as the character name
character_filtered = character_filtered.dropna(subset=['char_name'])
# drop the rows with the same character name
character_filtered = character_filtered.drop_duplicates(subset=['wiki_ID','char_name'])
character_filtered = character_filtered.set_index(['wiki_ID','char_name'])

display(character_filtered.head())

free_ID     release         DOB gender  \
wiki_ID char_name                                                              
975900  Akooshay                    /m/03vyhn  2001-08-24  1958-08-26      F   
        Lieutenant Melanie Ballard  /m/03vyhn  2001-08-24  1974-08-15      F   
        Desolation Williams         /m/03vyhn  2001-08-24  1969-06-15      M   
        Sgt Jericho Butler          /m/03vyhn  2001-08-24  1967-09-12      M   
        Bashira Kincaid             /m/03vyhn  2001-08-24  1977-09-25      F   

                                    height   ethnicity            act_name  \
wiki_ID char_name                                                            
975900  Akooshay                     1.620         NaN      Wanda De Jesus   
        Lieutenant Melanie Ballard   1.780  /m/044038p  Natasha Henstridge   
        Desolation Williams          1.727     /m/0x67            Ice Cube   
        Sgt Jericho Butler           1.750         NaN       Jason Statham   
        Bashira Kincaid              1.650         NaN         Clea DuVall   

                                    age_at_release free_char_map1  \
wiki_ID char_name                                                   
975900  Akooshay                              42.0     /m/0bgchxw   
        Lieutenant Melanie Ballard            27.0      /m/0jys3m   
        Desolation Williams                   32.0      /m/0jys3g   
        Sgt Jericho Butler                    33.0     /m/02vchl6   
        Bashira Kincaid                       23.0     /m/02vbb3r   

                                   free_char_map2 free_char_map3  
wiki_ID char_name                                                 
975900  Akooshay                       /m/0bgcj3x     /m/03wcfv7  
        Lieutenant Melanie Ballard     /m/0bgchn4      /m/0346l4  
        Desolation Williams            /m/0bgchn_     /m/01vw26l  
        Sgt Jericho Butler             /m/0bgchnq      /m/034hyc  
        Bashira Kincaid                /m/0bgchp9      /m/01y9xg

We drop all the columns related to FreeBase as we won't be using them

In [58]:
columns_to_drop = ['free_ID', 'release', 'ethnicity', 'free_char_map1', 'free_char_map2', 'free_char_map3']
columns_to_drop_existing = [col for col in columns_to_drop if col in character_filtered.columns]

character_filtered.drop(columns_to_drop_existing, axis=1, inplace=True)

display(character_filtered.head())

# Verify the indexes are unique
print(f"Is the indexing unique ? {character_filtered.index.is_unique}")

DOB gender  height  \
wiki_ID char_name                                               
975900  Akooshay                    1958-08-26      F   1.620   
        Lieutenant Melanie Ballard  1974-08-15      F   1.780   
        Desolation Williams         1969-06-15      M   1.727   
        Sgt Jericho Butler          1967-09-12      M   1.750   
        Bashira Kincaid             1977-09-25      F   1.650   

                                              act_name  age_at_release  
wiki_ID char_name                                                       
975900  Akooshay                        Wanda De Jesus            42.0  
        Lieutenant Melanie Ballard  Natasha Henstridge            27.0  
        Desolation Williams                   Ice Cube            32.0  
        Sgt Jericho Butler               Jason Statham            33.0  
        Bashira Kincaid                    Clea DuVall            23.0

Is the indexing unique ? True


## New dataframe character name

lets create a new dataframe containing the character names with their respective movie.

In [59]:
name_df = character_filtered.reset_index()[['wiki_ID', 'char_name']].copy(deep=True)
display(name_df.head())

,wiki_ID,char_name
0,975900,Akooshay
1,975900,Lieutenant Melanie Ballard
2,975900,Desolation Williams
3,975900,Sgt Jericho Butler
4,975900,Bashira Kincaid


Lets split the character names and explode it

In [60]:
# Split the character names into words and explode the lists
exploded_name_df = name_df.assign(char_words=name_df['char_name'].str.split()).explode('char_words')
word_name_df = exploded_name_df.drop(columns=['char_name'])

word_name_df.head()

,wiki_ID,char_words
0,975900,Akooshay
1,975900,Lieutenant
1,975900,Melanie
1,975900,Ballard
2,975900,Desolation


### Filter the names to keep only the ones available in the baby name dataset

In [61]:
# does the word_name_df has duplicates?
duplicates = word_name_df.duplicated()
print(f"number of duplicates in word_name_df = {duplicates.sum()}")
word_name_df = word_name_df.drop_duplicates()
duplicates_after = word_name_df.duplicated()
print(f"number of duplicates in word_name_df = {duplicates_after.sum()}")

number of duplicates in word_name_df = 28595
number of duplicates in word_name_df = 0


In [62]:
baby_name_only_df = baby_name_df[['name']].copy(deep=True)

# does the baby_name_only_df has duplicates?
duplicates = baby_name_only_df.duplicated()
print(f"number of duplicates in baby_name_only_df = {duplicates.sum()}")
# drop these duplicates
baby_name_only_df = baby_name_only_df.drop_duplicates()
duplicates = baby_name_only_df.duplicated()
print(f"number of duplicates in baby_name_only_df = {duplicates.sum()}")

print("\nbaby_name_only_df :")
display(baby_name_only_df.head())

number of duplicates in baby_name_only_df = 1982709
number of duplicates in baby_name_only_df = 0

baby_name_only_df :


,name
0,Mary
1,Anna
2,Emma
3,Elizabeth
4,Minnie


In [63]:
# Use pd.merge to filter rows based on 'names' column
word_name_filtered_df = pd.merge(word_name_df.reset_index(), baby_name_only_df, left_on='char_words', right_on='name', how='inner')
# word_name_filtered_df = word_name_filtered_df.drop_duplicates()
# word_name_filtered_df.set_index(['wiki_ID', 'char_words'], inplace=True)
word_name_filtered_df.drop(columns=['index', 'name'], inplace=True)
word_name_filtered_df.set_index(['wiki_ID', 'char_words'], inplace=True)
print("word_name_filtered_df :")
display(word_name_filtered_df.head())

# Verify the indexes are unique
print(f"Is the indexing unique ? {word_name_filtered_df.index.is_unique}")

word_name_filtered_df :


,
wiki_ID,char_words
975900,Lieutenant
7668793,Lieutenant
24226493,Lieutenant
3388805,Lieutenant
8231713,Lieutenant


Is the indexing unique ? True


In [64]:
# check for the first movie of the CMU dataset, seems ok
word_name_filtered_df.loc[975900]

""
char_words
Lieutenant
Melanie
Ballard
Williams
Jericho
Butler
Bashira
Kincaid
Helena


The name Lieutenant is still in the filtered dataframe. Let's check if this name is in the baby name dataset

In [65]:
name_to_search = 'Lieutenant'

# Check if the name is present
is_name_present = name_to_search in baby_name_only_df['name'].values

if is_name_present:
    print(f"{name_to_search} is present in the DataFrame.")
else:
    print(f"{name_to_search} is not present in the DataFrame.")

Lieutenant is present in the DataFrame.


## Exportation

In [85]:
folder_processed_data_path = './processed_data/'

In [86]:
display(movie_rating_df.head())
# Export DataFrame to a CSV file in the processed data folder
movie_rating_df.to_csv(os.path.join(folder_processed_data_path, 'movie_df.csv'), index=False)

,index,wiki_ID,free_ID,mov_name,release,revenue,runtime,languages,countries,genres,averageRating,numVotes
0,0,975900,/m/03vyhn,Ghosts of Mars,2001,14010832.0,98.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/01jfsb"": ""Thriller"", ""/m/06n90"": ""Science...",4.9,56880
1,1,3196793,/m/08yl5d,Getting Away with Murder: The JonBenét Ramsey ...,2000,NaN,95.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/02n4kr"": ""Mystery"", ""/m/03bxz7"": ""Biograp...",6.0,69
2,2,28463795,/m/0crgdbh,Brun bitter,1988,NaN,83.0,"{""/m/05f_3"": ""Norwegian Language""}","{""/m/05b4w"": ""Norway""}","{""/m/0lsxr"": ""Crime Fiction"", ""/m/07s9rl0"": ""D...",5.6,40
3,3,9363483,/m/0285_cd,White Of The Eye,1987,NaN,110.0,"{""/m/02h40lc"": ""English Language""}","{""/m/07ssc"": ""United Kingdom""}","{""/m/01jfsb"": ""Thriller"", ""/m/0glj9q"": ""Erotic...",6.1,2891
4,4,261236,/m/01mrr1,A Woman in Flames,1983,NaN,106.0,"{""/m/04306rv"": ""German Language""}","{""/m/0345h"": ""Germany""}","{""/m/07s9rl0"": ""Drama""}",5.9,623


In [87]:
display(character_filtered.reset_index().head())
# Export DataFrame to a CSV file in the processed data folder
character_filtered.reset_index().to_csv(os.path.join(folder_processed_data_path, 'character_df.csv'), index=False)

,wiki_ID,char_name,DOB,gender,height,act_name,age_at_release
0,975900,Akooshay,1958-08-26,F,1.620,Wanda De Jesus,42.0
1,975900,Lieutenant Melanie Ballard,1974-08-15,F,1.780,Natasha Henstridge,27.0
2,975900,Desolation Williams,1969-06-15,M,1.727,Ice Cube,32.0
3,975900,Sgt Jericho Butler,1967-09-12,M,1.750,Jason Statham,33.0
4,975900,Bashira Kincaid,1977-09-25,F,1.650,Clea DuVall,23.0


In [88]:
display(word_name_filtered_df.reset_index().head())
# Export DataFrame to a CSV file in the processed data folder
word_name_filtered_df.reset_index().to_csv(os.path.join(folder_processed_data_path, 'name_by_movie_df.csv'), index=False)

,wiki_ID,char_words
0,975900,Lieutenant
1,7668793,Lieutenant
2,24226493,Lieutenant
3,3388805,Lieutenant
4,8231713,Lieutenant


In [89]:
display(baby_name_with_percentage_df.reset_index().head())
# Export DataFrame to a CSV file in the processed data folder
baby_name_with_percentage_df.reset_index().to_csv(os.path.join(folder_processed_data_path, 'baby_name_df.csv'), index=False)

,name,year,number,percentage
0,Aaban,2007,5,0.000125
1,Aadam,2007,8,0.000200
2,Aadan,2007,8,0.000200
3,Aadarsh,2007,13,0.000325
4,Aaden,2007,157,0.003928


In [90]:
display(rating_df.reset_index().head())
# Export DataFrame to a CSV file in the processed data folder
rating_df.reset_index().to_csv(os.path.join(folder_processed_data_path, 'rating_df.csv'), index=False)

,tconst,averageRating,numVotes
0,tt0000001,5.7,2004
1,tt0000002,5.8,269
2,tt0000003,6.5,1904
3,tt0000004,5.5,178
4,tt0000005,6.2,2685
